# ML model generation

This notebook creates and trains a simple fully connected model using MNIST dataset. At the end, the model is exported as `model.bin`.

In [9]:
#|export
import torch
from torch import nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
#|export
def generate_dataloader(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to fit ResNet input size
        transforms.ToTensor(),
    ])

    train_dataset = ImageFolder(root=f"data/imagenette2/train", transform=transform)
    test_dataset = ImageFolder(root=f"data/imagenette2/val", transform=transform)

    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             num_workers=torch.get_num_threads())
    testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=torch.get_num_threads())

    return trainloader, testloader

In [11]:
#|export
def test_model(model, testloader):
    loss_fn = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        vloss = 0.
        correct = 0.

        # Fetch the first batch
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            vloss += loss_fn(outputs, targets).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted==targets).sum().item()

    return vloss/len(testloader),  correct/len(testloader.dataset)

def train_model(model):  
    # training
    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=0.001)
    trainloader, testloader = generate_dataloader()
    for epoch in range(1):

        model.train()
        tloss = 0
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            opt.zero_grad()
            out = model(inputs)
            loss = loss_fn(out, targets)
            loss.backward()
            tloss += loss.item()
            opt.step()
        tloss = tloss/len(trainloader)
        vloss, correct = test_model(model, testloader)

        print('LOSS train {} valid {} accuracy {:.5f}'.format(tloss, vloss, correct))
    torch.save(model, "model.pt")

In [5]:
import torch
seed = 1337
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [6]:
trainloader, testloader = generate_dataloader()

In [7]:
%%time
from export import export_model
from model import Model

model = Model()
train_model(model)
export_model(model)

In [8]:
# Tests for the above function(s)

loss, acc = test_model(torch.load("model.pt"), testloader)
print(f"Loss: {loss:.4f} Accuracy: {100*acc:.2f} %")

assert loss - 0.11901 < 0.0001
assert acc - 0.97410 < 0.00001

Loss: 0.1190 Accuracy: 97.30 %
